In [3]:
import requests

URL = "https://api.hyperliquid.xyz/info"

payload = {
    "type": "allMids"
}

resp = requests.post(URL, json=payload, timeout=10)
resp.raise_for_status()

mids = resp.json()

print(f"BTC mid price: {mids.get('BTC')}")
print(f"ETH mid price: {mids.get('ETH')}")


BTC mid price: 86685.5
ETH mid price: 2936.15


### Fetch prices

In [2]:
import requests

def fetch_mids():
    url = "https://api.hyperliquid.xyz/info"
    payload = {"type": "allMids"}
    r = requests.post(url, json=payload)
    r.raise_for_status()
    return r.json()


### Maintain rolling price history

In [3]:
from collections import deque

price_history = {
    "BTC": deque(maxlen=50),
    "ETH": deque(maxlen=50)
}


### Update Prices

In [4]:
mids = fetch_mids()

price_history["BTC"].append(float(mids["BTC"]))
price_history["ETH"].append(float(mids["ETH"]))


### Compute SMA

In [5]:
import numpy as np

def compute_sma(prices):
    if len(prices) == 0:
        return None
    return np.mean(prices)


### Signal

In [6]:
def generate_signal(price, sma):
    if sma is None:
        return 0
    return 1 if price > sma else 0


### Compute signals for BTC & ETH

In [7]:
signals = {}

for asset in ["BTC", "ETH"]:
    price = price_history[asset][-1]
    sma = compute_sma(price_history[asset])
    signals[asset] = generate_signal(price, sma)

signals


{'BTC': 0, 'ETH': 0}

### Trade Engine

In [8]:
state = {
    "BTC": {"position": 0, "entry_price": None, "pnl": 0.0, "trades": 0},
    "ETH": {"position": 0, "entry_price": None, "pnl": 0.0, "trades": 0},
}


In [9]:
def execute_trade(asset, target_pos, price, state, max_position_usd):
    current_pos = state[asset]["position"]
    
    if current_pos == target_pos:
        return None  # no trade
    
    # position size in units
    size = max_position_usd / price
    
    trade = {
        "asset": asset,
        "price": price,
        "direction": "BUY" if target_pos == 1 else "SELL",
        "size": size
    }
    
    # Enter long
    if current_pos == 0 and target_pos == 1:
        state[asset]["position"] = 1
        state[asset]["entry_price"] = price
        state[asset]["trades"] += 1
    
    # Exit long
    elif current_pos == 1 and target_pos == 0:
        entry = state[asset]["entry_price"]
        pnl = (price - entry) * size
        state[asset]["pnl"] += pnl
        state[asset]["position"] = 0
        state[asset]["entry_price"] = None
        state[asset]["trades"] += 1
        trade["realized_pnl"] = pnl
    
    return trade


### Run Execution

In [10]:
trades = []

limits = {"BTC": 5000, "ETH": 3000}

for asset in ["BTC", "ETH"]:
    trade = execute_trade(
        asset=asset,
        target_pos=signals[asset],
        price=price_history[asset][-1],
        state=state,
        max_position_usd=limits[asset]
    )
    if trade:
        trades.append(trade)

trades


[]

### Logging

In [11]:
import pandas as pd
from datetime import datetime

log_rows = []

def log_trade(trade, signal, position):
    row = {
        "timestamp": datetime.utcnow().isoformat(),
        "asset": trade["asset"],
        "price": trade["price"],
        "signal": signal,
        "direction": trade["direction"],
        "size": trade["size"],
        "position": position,
        "realized_pnl": trade.get("realized_pnl", 0.0),
    }
    log_rows.append(row)


In [18]:
from datetime import datetime
import time

def status_print(asset, price, sma, signal, position, trade=None):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    
    sma_str = f"{sma:.2f}" if sma is not None else "NA"
    
    msg = (
        f"[{ts}] {asset} | "
        f"Price: {price:.2f} | "
        f"SMA: {sma_str} | "
        f"Signal: {'LONG' if signal == 1 else 'FLAT'} | "
        f"Position: {position}"
    )
    
    if trade:
        msg += f" | TRADE: {trade['direction']} {trade['size']:.4f}"
    
    print(msg)


### Log executed trades

In [19]:
for trade in trades:
    asset = trade["asset"]
    log_trade(
        trade=trade,
        signal=signals[asset],
        position=state[asset]["position"]
    )

pd.DataFrame(log_rows)


""


### Wrap in a loop (scheduler)

In [20]:
for _ in range(10):
    mids = fetch_mids()
    
    for asset in ["BTC", "ETH"]:
        price_history[asset].append(float(mids[asset]))
    
    for asset in ["BTC", "ETH"]:
        price = price_history[asset][-1]
        sma = compute_sma(price_history[asset])
        signal = generate_signal(price, sma)
        
        trade = execute_trade(
            asset, signal, price, state, limits[asset]
        )
        
        status_print(
            asset=asset,
            price=price,
            sma=sma,
            signal=signal,
            position=state[asset]["position"],
            trade=trade
        )
        
        if trade:
            log_trade(trade, signal, state[asset]["position"])
    
    print("-" * 80)
    time.sleep(60)


[09:42:19] BTC | Price: 86394.50 | SMA: 86372.00 | Signal: LONG | Position: 1 | TRADE: BUY 0.0579
[09:42:19] ETH | Price: 2924.75 | SMA: 2924.00 | Signal: LONG | Position: 1 | TRADE: BUY 1.0257
--------------------------------------------------------------------------------
[09:43:19] BTC | Price: 86383.50 | SMA: 86374.30 | Signal: LONG | Position: 1
[09:43:19] ETH | Price: 2924.75 | SMA: 2924.15 | Signal: LONG | Position: 1
--------------------------------------------------------------------------------
[09:44:20] BTC | Price: 86402.50 | SMA: 86379.00 | Signal: LONG | Position: 1
[09:44:20] ETH | Price: 2927.25 | SMA: 2924.67 | Signal: LONG | Position: 1
--------------------------------------------------------------------------------
[09:45:20] BTC | Price: 86402.50 | SMA: 86382.36 | Signal: LONG | Position: 1
[09:45:20] ETH | Price: 2926.85 | SMA: 2924.98 | Signal: LONG | Position: 1
--------------------------------------------------------------------------------
[09:46:20] BTC | Pri